In [1]:
import findspark
findspark.init()

In [2]:
"""
Spark Database
Single, local database. NOt shared across multiple notebooks, not shared across worker. 
Built into spark working director, Generally used for simple development/learning purpose

In production, we will be using Hive Data Catalog
"""

'\nSpark Database\nSingle, local database. NOt shared across multiple notebooks, not shared across worker. \nBuilt into spark working director, Generally used for simple development/learning purpose\n\nIn production, we will be using Hive Data Catalog\n'

In [ ]:
"""
Spark Local Database
Only for dev only, not for production

3 components involved

1) metadata - database name, tables, columns, data types, location where the data is stored,
    managed by HIVE, HIVE internaly uses derby db to store all metadata
2) spark temporary location spark.local.dir, /home/ubuntu/spark-temp where temp data used internally is stored
3) spark data warehouse where all the database data will be stored.
    We can see database, tables, there data with metadata, 
"""

In [5]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkDatabase")

# embedded, simple, LOCAL spark database/warehouse
# spark will store tempoary files
# enable hive support for sql database
# enable hiveSupport Hive catalog to be embedded inside working directory
config.set("spark.local.dir", "/home/ubuntu/spark-temp")
# spark data (not meta) does into /home/ubuntu/spark-warehouse
config.set("spark.sql.warehouse.dir", "/home/ubuntu/spark-warehouse")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame

# enableHiveSupport() create a meta catalog/database using derby database
# inside current working directory
# multiple notebooks cant share at same time
# inside pyspark-notebooks, you could see metastore_db
# metastore will have meta data: database, tables, columns, data types
# data located in hdfs or files system or s3
# derby.Log - derby database log

spark = SparkSession.builder\
                    .config(conf=config)\
                    .enableHiveSupport()\
                    .getOrCreate()

sc = spark.sparkContext

In [11]:
df = spark.sql("SHOW DATABASES")
df.show()

+------------+
|databaseName|
+------------+
|     default|
|stocklocaldb|
+------------+



In [7]:
spark.sql("CREATE DATABASE IF NOT EXISTS stocklocaldb")

22/03/07 20:14:07 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
22/03/07 20:14:07 WARN ObjectStore: Failed to get database stocklocaldb, returning NoSuchObjectException


DataFrame[]

In [10]:
# Craete spark Managed table
# insert, update ( delete will not work at 2.x)
# need to type spark.sql to managed data
spark.sql("CREATE TABLE IF NOT EXISTS stocks(symbol STRING, industry STRING)")

22/03/07 20:15:09 WARN HiveMetaStore: Location: file:/home/ubuntu/spark-warehouse/stocks specified for non-external table:stocks


DataFrame[]

In [12]:
spark.sql("""
    INSERT INTO stocks VALUES('INFY', 'IT')
"""
         )

DataFrame[]

In [14]:
spark.sql("SELECT * FROM  stocks").show()

+------+--------+
|symbol|industry|
+------+--------+
|  INFY|      IT|
+------+--------+



In [15]:
spark.sql("DROP TABLE IF EXISTS stocks")

DataFrame[]

In [16]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [17]:
spark.sql("DROP DATABASE IF EXISTS stocklocaldb")

DataFrame[]

In [19]:
spark.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [20]:
spark.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [22]:
# create DATABASE called prodcudb
spark.sql("CREATE DATABASE IF NOT EXISTS productdb")

22/03/07 20:40:29 WARN ObjectStore: Failed to get database productdb, returning NoSuchObjectException


DataFrame[]

In [32]:
# create a table called products with 3 fields (id int, name string, price int)
# create table prodcutdb.products
spark.sql("CREATE TABLE IF NOT EXISTS productdb.products(id INT, product_name STRING, price INT)")

DataFrame[]

In [35]:
# insert 2 recors into prodcutsdb.prodcuts
spark.sql("""
    INSERT INTO productdb.products VALUES(1, "Water", 2), 
                                (2, "OJ", 3)
"""
         )

DataFrame[]

In [37]:
spark.sql("SELECT * FROM productdb.products").show()

+---+-----+-----+
| id| name|price|
+---+-----+-----+
|  1|Water|    2|
|  2|   OJ|    3|
+---+-----+-----+



In [42]:
spark.sql("SELECT * FROM productdb.products WHERE price <= 2").show()

+---+-----+-----+
| id| name|price|
+---+-----+-----+
|  1|Water|    2|
+---+-----+-----+

